# Editing Tap on/off data

The tap on/off data is an amalgam of train, bus, ferry and light rail data with locations ranging from postcodes to street and station names as well as names of buildings. Without initially addressing this issue, no actual progress can be made towards developing a heat map using GeoPandas. Thus, this notebook creates a diuctionary which can be used to convert all the locations into strictly postcode data.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# read the offical NSW transport 
time_loc_1 = pd.read_csv("files/time-loc_16NOV.csv")
time_loc_2 = pd.read_csv("files/time-loc_23MAR.csv")
time_loc_3 = pd.read_csv("files/time-loc_24AUG.csv")
time_loc_4 = pd.read_csv("files/time-loc_24FEB.csv")
time_loc_1.head()

,mode,date,tap,time,loc,count
0,bus,20201116,off,0:15,Sydney CBD_Wynyard Station_George St(57),19
1,bus,20201116,off,0:45,Bathurst St and Elizabeth St(116),21
2,bus,20201116,off,2:15,Sydney TAFE(144),19
3,bus,20201116,off,3:15,Sydney CBD_Defence Plaza(105),22
4,bus,20201116,off,4:00,2140,19


In [4]:
## print the keys for each datasets given.
print(time_loc_1.keys())
print(time_loc_2.keys())
print(time_loc_3.keys())
print(time_loc_4.keys())
## All the dataset are clean to work with. 

Index(['mode', 'date', 'tap', 'time', 'loc', 'count'], dtype='object')
Index(['mode', 'date', 'tap', 'time', 'loc', 'count'], dtype='object')
Index(['mode', 'date', 'tap', 'time', 'loc', 'count'], dtype='object')
Index(['mode', 'date', 'tap', 'time', 'loc', 'count'], dtype='object')


### Dictionary:
The following creates a dictionary for all the postcodes and respective areas so the four dataframes can be altered with faster computation speed.

In [5]:
pcd = pd.read_csv("files/postcode_data.csv")
pcd.head()

,Postcode,Area
0,2000,Darling Harbour
1,2000,Dawes Point
2,2000,Haymarket
3,2000,Millers Point
4,2000,Parliament House


In [6]:
pc_dict = dict()
for i in range(1,len(pcd['Postcode'])):
    name = str(pcd.iloc[i,1])
    id = pcd.iloc[i,0]
    pc_dict[name]=id

Although exceedingly ugly, this chunk of code renames 142 different areas which did not have an easily accessible name to convert into a postcode. While there is probably a way of having code read a google maps postcode value to determine the postcode for the street intersections and other easier to find locations, 142 entries is not too challenging to check by hand.
Furthermore, it allowed us to identify places which are irrelevant to the scope of this project. For example, Cockatoo Island. Our data contains ferry data as well and as such this stood out as a place which did not require a postcode for analysis, as such it was set to -1, like the unknown values.

In [7]:
pc_dict['Bathurst St and Elizabeth St']=2000   
pc_dict['Sydney TAFE']=2007
pc_dict['Prince Alfred Park']=2010
pc_dict['King St and George St']=2000
pc_dict['National Art School']=2010
pc_dict['Central']=2000
pc_dict['Market St and Kent St']=2000
pc_dict['News Ltd']=2190
pc_dict['Albion St and Flinders St']=2010
pc_dict['Garden Island']=2011
pc_dict['University of Sydney_Main Camperdown Campus']=2006
pc_dict['Westpac Building']=2000
pc_dict['Sheraton On The Park']=2000
pc_dict['St James']=2000
pc_dict['Centenary Square and George']=2150
pc_dict['Town Hall']=2000
pc_dict['Allianz Centre']=2000
pc_dict['UTS_Ultimo_West']=2007
pc_dict['Market St and Clarence St']=2000
pc_dict['Camperdown Park']=2050
pc_dict['Oxford Square']=2021
pc_dict["St Vincent's Caritas Centre"]=2010
pc_dict['Fish Markets MLR']=2009
pc_dict['Surry Hills Shopping Village']=2010
pc_dict['Paddington_Flinders St and Taylor St']=2010
pc_dict['Forest Lodge_Arundel St and Short St']=2037
pc_dict['Liverpool St and Nithsdale St']=2000
pc_dict['Queen Victoria Building']=2000
pc_dict['Lee St_Sydney']=2000
pc_dict['Kingsgate Shopping Centre']=2010
pc_dict['Redfern_Baptist St and Zamia St']=2016
pc_dict['Annandale_Johnston St and Booth St']=2038
pc_dict['United Dental Hospital Of Sydney']=2000
pc_dict['King St and Clarence St']=2000
pc_dict['Wentworth Park']=2037
pc_dict['Druitt St and Kent St']=2000
pc_dict['Park St and Elizabeth St']=2000
pc_dict['Martin Place and Elizabeth St']=2000
pc_dict['Bourke St and Arthur St']=2010
pc_dict['ABC Ultimo Centre']=2007
pc_dict['John St Square MLR']=2009
pc_dict['Bourke St and Mort St']=2010
pc_dict['Wentworth Park MLR']=2037
pc_dict['Darling Harbour_Crown Plaza Hotel']=2000
pc_dict["St Luke's Hospital"]=2010
pc_dict['Rydges Camperdown']=2050
pc_dict["St Vincent's Hospital"]=2010
pc_dict['Sydney Police Centre']=2010
pc_dict['Barangaroo, 2']=2000
pc_dict['Circular Quay']=2000
pc_dict['Queens']=2000
pc_dict['McMahons Point']=2060
pc_dict['Taronga Zoo']=2088
pc_dict['Mosman Bay']=2088
pc_dict['Sydney Olympic Park']=2127
pc_dict['Kurraba Point']=2089
pc_dict['South Mosman']=2088
pc_dict['Pyrmont Bay']=2009
pc_dict['Central Chalmers Street']=2000
pc_dict['UNSW High Street']=2083
pc_dict['Central Grand Concourse']=2000
pc_dict['Wynyard']=2000
pc_dict['Fish Market']=2009
pc_dict['QVB']=2000
pc_dict['Bridge Street']=2000
pc_dict['Capitol Square']=2000
pc_dict['Chinatown']=2000
pc_dict['Civic']=2000
pc_dict['The Star']=2009
pc_dict['John Street Square']=2009
pc_dict['Jubilee Park']=2037
pc_dict['Convention']=2000
pc_dict['Honeysuckle']=2300
pc_dict['UNSW Anzac Parade']=2052
pc_dict['Exhibition Centre']=2000
pc_dict["Paddy's Markets"]=2000
pc_dict['Newcastle Beach']=2300
pc_dict['Newcastle Interchange']=2300
pc_dict['Juniors Kingsford']=2032
pc_dict['Leichhardt North']=2040
pc_dict['UNKNOWN']=-1                           # Unknown is -1
pc_dict['Royal Randwick']=2031
pc_dict['ES Marks']=2033
pc_dict['Arlington']=2203
pc_dict['Lewisham West']=2049
pc_dict['Wansey Road']=2031
pc_dict['Hawthorne']=2045
pc_dict['Taverners Hill']=2049
pc_dict['Dulwich Grove']=2203
pc_dict['Waratah Mills']=2203
pc_dict['Crown Street']=2010
pc_dict['Rozelle Bay']=2038
pc_dict['Flemington']=2140
pc_dict['Martin Place']=2000
pc_dict['Museum']=2000
pc_dict['Green Square']=2015
pc_dict['Domestic']=2020
pc_dict['Olympic Park']=2127
pc_dict['Clyde']=2142
pc_dict['Leightonfield']=2163
pc_dict['Tallawong']=2155
pc_dict['International']=2020
pc_dict['Macdonaldtown']=2015
pc_dict['Macarthur']=2560
pc_dict['North Wollongong']=2500
pc_dict['Norwest']=2153
pc_dict['Hills Showground']=2154
pc_dict['Shellharbour Junction']=2529
pc_dict['Hawkesbury River']=2083
pc_dict['Victoria Street']=2323
pc_dict['Strawberry Hills_Elizabeth St and Belvoir St']=2010
pc_dict['Barangaroo, 1']=2000
pc_dict['Convention Centre']=2000
pc_dict['East Richmond']=2753
pc_dict['Darling Island_Pyremont Park']=2009
pc_dict['Campbell St and Mary St']=2010
pc_dict['Experiment St']=2009
pc_dict['City Star Casino']=2009
pc_dict['Greenwich Point']=2065
pc_dict['Cockatoo Island']=-1                      # Cockatoo Island is -1
pc_dict['Cook and Phillip Park']=2000
pc_dict['Campbell St and Mary St']=2010
pc_dict['Powerhouse Museum Ultimo']=2007
pc_dict['Redfern Mail Exchange Redfern']=2016
pc_dict['East Sydney High School']=2010
pc_dict['Redfern Oval']=2016
pc_dict['Forest Lodge Public School']=2037
pc_dict['Elizabeth Bay House']=2011
pc_dict['Martin Place and Phillip St']=2000
pc_dict['Centenary Square']=2150
pc_dict['Piccadilly Arcade']=2035
pc_dict["St. Mary's Cathedral"]=2000
pc_dict['Royal Botanic Gardens']=2000
pc_dict['Royal Prince Alfred Hospital']=2050
pc_dict['Rozelle Bay MLR']=2039
pc_dict['CUB Site Chippendale_South']=2008
pc_dict['Sydney Aquarium']=2000
pc_dict['Goulburn St and Wentworth St']=2000
pc_dict['Kissing Point']=2112
pc_dict['Old Cremorne']=2090
pc_dict['Marion']=2040
pc_dict['Cockle Creek']=2284

After mapping initially in GeoPandas, the entry 'Punchbowl' was 2460, which was incorrect, the actual postcode for this value is 2196.

In [31]:
pc_dict['Punchbowl']=2196
pc_dict['Punchbowl']

2196

Three postcode values did not have associated geopandas files, when inspected, it is because they correspond to alternate postcodes for areas:
1. 2057 is for Chatswood, which instead should be 2067
2. 2520 is for Wollongong, which should be 2500
3. 2751 is for Penrith, which should be 2750

These changes were found in the Geopandas Heatmap code when it failed to map properly and have been implemetned in the function below

To generate this dictionary, the 142 extra locations were found using the follow code. 
This is now obselete as new files of the tap on/off data were created with only postcode data.

The following alters some of the more common names of train and bus stops as well as the ferry and light rail data. It is also used to find the outlying locations which aren't changed to postcodes in the first pass due to having obscure/unknown locations/names.
Finally, after the first time running each of the dataframes through this code, all the extra cases were accounted for, once they are run through this again, the location data is only postcodes.

In [32]:
def nametopostcode(time_loc):
    for i in range(0,len(time_loc['loc'])):
        if str(time_loc.iloc[i,4]).isnumeric() == False:
            if 'Glebe' in time_loc.iloc[i,4]:
                time_loc.iloc[i,4]='Glebe'
            if 'Sydney CBD' in time_loc.iloc[i,4]:
                time_loc.iloc[i,4]='Sydney'
            if 'Darlinghurst' in time_loc.iloc[i,4]:
                time_loc.iloc[i,4]='Darlinghurst'
            if ' Station' in time_loc.iloc[i,4]:
                time_loc.iloc[i,4] = time_loc.iloc[i,4].replace(' Station','')
            if ' Light Rail' in time_loc.iloc[i,4]:
                time_loc.iloc[i,4] = time_loc.iloc[i,4].replace(' Light Rail','')
            if ' Wharf' in time_loc.iloc[i,4]:
                time_loc.iloc[i,4] = time_loc.iloc[i,4].replace(' Wharf','')
            if '(' in time_loc.iloc[i,4]:
                end = time_loc.iloc[i,4].index('(')
                time_loc.iloc[i,4] = time_loc.iloc[i,4][0:end]
                
            if str(time_loc.iloc[i,4]).isnumeric() == False:
                if time_loc.iloc[i,4] in pc_dict.keys():
                    time_loc.iloc[i,4] = pc_dict[time_loc.iloc[i,4]]

In [33]:
time_loc_4

,mode,date,tap,time,loc,count
0,bus,20200224,off,0:00,2022,21
1,bus,20200224,off,0:00,2031,22
2,bus,20200224,off,0:15,2026,23
3,bus,20200224,off,0:15,2035,19
4,bus,20200224,off,0:15,Bathurst St and Elizabeth St(116),28
...,...,...,...,...,...,...
276763,unknown,20200227,off,20:15,UNKNOWN,31
276764,unknown,20200227,on,6:30,UNKNOWN,24
276765,unknown,20200227,on,17:00,UNKNOWN,55
276766,unknown,20200227,on,20:00,UNKNOWN,27


In [34]:
nametopostcode(time_loc_4)
time_loc_4

,mode,date,tap,time,loc,count
0,bus,20200224,off,0:00,2022,21
1,bus,20200224,off,0:00,2031,22
2,bus,20200224,off,0:15,2026,23
3,bus,20200224,off,0:15,2035,19
4,bus,20200224,off,0:15,2000,28
...,...,...,...,...,...,...
276763,unknown,20200227,off,20:15,-1,31
276764,unknown,20200227,on,6:30,-1,24
276765,unknown,20200227,on,17:00,-1,55
276766,unknown,20200227,on,20:00,-1,27


In [35]:
time_loc_4['loc'].unique()

array(['2022', '2031', '2026', '2035', 2000, '2140', '2216', '2135',
       '2020', '2560', '2170', '2770', 2007, '2134', '2148', '2150',
       '2259', '2760', 2010, '2113', '2138', '2164', '2165', '2250',
       '2750', '2766', 2190, '2019', '2100', '2145', '2147', '2200',
       '2212', '2220', '2256', 2016, '2018', '2021', '2033', '2034',
       '2039', '2046', '2067', '2088', '2089', '2095', '2107', '2127',
       '2136', '2142', '2144', '2153', '2155', '2217', 2009, 2006, '2032',
       '2060', '2065', '2099', '2103', '2121', '2128', '2151', '2154',
       '2167', '2190', '2194', '2756', '2762', 2150, '-1', '2015', '2017',
       '2029', '2030', '2036', '2042', '2047', '2066', '2074', '2077',
       '2086', '2090', '2093', '2097', '2101', '2112', '2114', '2116',
       '2122', '2131', '2141', '2160', '2166', '2176', '2204', '2208',
       '2210', '2211', '2500', '2565', '2566', '2765', '2023', '2024',
       '2027', '2075', '2076', '2085', '2111', '2115', '2161', '2162',
       '

In [36]:
nonnum = list() # non-numerics to find all extra cases to account for
for i in range(0,len(time_loc_4['loc'])): # this was changed to the different dataframes so each was parsed one time
    if str(time_loc_4.iloc[i,4]).isnumeric() == False:
        nonnum.append(time_loc_4.iloc[i,4])
output = []
for x in nonnum:
    if x not in output:
        output.append(x)
output

['-1', -1]

After running this for the last time, all non-numerics are -1 since the minus sign isn't recognised as such.

In [37]:
nametopostcode(time_loc_1)
nametopostcode(time_loc_2)
nametopostcode(time_loc_3)
# nametopostcode(time_loc_4), was done above

In [46]:
# updates from after code was written due to postcodes being wrong and not have geopandas data
def wrongpostcodes(time_loc):
    for i in range(0,len(time_loc['loc'])):
        if time_loc.iloc[i,4] == '2057':
            time_loc.iloc[i,4] = 2067
        if time_loc.iloc[i,4] == 2520:
            time_loc.iloc[i,4] = 2500
        if time_loc.iloc[i,4] == 2751:
            time_loc.iloc[i,4] = 2750
            
# Strangely, only 2057 was in a string, the other two were numbers.

In [47]:
wrongpostcodes(time_loc_1)
wrongpostcodes(time_loc_2)
wrongpostcodes(time_loc_3)
wrongpostcodes(time_loc_4)

In [48]:
time_loc_1.to_csv('pc_time-loc_16NOV.csv')
time_loc_2.to_csv('pc_time-loc_23MAR.csv')
time_loc_3.to_csv('pc_time-loc_24AUG.csv')
time_loc_4.to_csv('pc_time-loc_24FEB.csv')

Once these files were created, they were moved into the Github files to be used for the rest of the project.